# Imports e dependências

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

import pickle

from collections import Counter

# Carregamento de dados e formatação para uso

### Obter dados do arquivo

In [3]:
# Dados para treino
dados = pd.read_csv('Dados/titanic.csv', sep=';', na_values=["", " ", "NA", "N/A"])

In [4]:
dados

,"PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked"
0,"1,0,3,""Braund, Mr. Owen Harris"",male,22,1,0,A/..."
1,"2,1,1,""Cumings, Mrs. John Bradley (Florence Br..."
2,"3,1,3,""Heikkinen, Miss. Laina"",female,26,0,0,S..."
3,"4,1,1,""Futrelle, Mrs. Jacques Heath (Lily May ..."
4,"5,0,3,""Allen, Mr. William Henry"",male,35,0,0,3..."
...,...
886,"887,0,2,""Montvila, Rev. Juozas"",male,27,0,0,21..."
887,"888,1,1,""Graham, Miss. Margaret Edith"",female,..."
888,"889,0,3,""Johnston, Miss. Catherine Helen """"Car..."
889,"890,1,1,""Behr, Mr. Karl Howell"",male,26,0,0,11..."


### Separação dos dados em colunas

In [5]:
dados = dados['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked'].str.split(',', expand=True)

In [6]:
dados

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,0,3,"""Braund","Mr. Owen Harris""",male,22,1,0,A/5 21171,7.25,,S
1,2,1,1,"""Cumings","Mrs. John Bradley (Florence Briggs Thayer)""",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"""Heikkinen","Miss. Laina""",female,26,0,0,STON/O2. 3101282,7.925,,S
3,4,1,1,"""Futrelle","Mrs. Jacques Heath (Lily May Peel)""",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"""Allen","Mr. William Henry""",male,35,0,0,373450,8.05,,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"""Montvila","Rev. Juozas""",male,27,0,0,211536,13,,S
887,888,1,1,"""Graham","Miss. Margaret Edith""",female,19,0,0,112053,30,B42,S
888,889,0,3,"""Johnston","Miss. Catherine Helen """"Carrie""""""",female,,1,2,W./C. 6607,23.45,,S
889,890,1,1,"""Behr","Mr. Karl Howell""",male,26,0,0,111369,30,C148,C


In [7]:

dados.columns = ['PassengerId', 'Survived', 'Pclass', 'Surname', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

In [8]:
dados

,PassengerId,Survived,Pclass,Surname,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"""Braund","Mr. Owen Harris""",male,22,1,0,A/5 21171,7.25,,S
1,2,1,1,"""Cumings","Mrs. John Bradley (Florence Briggs Thayer)""",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"""Heikkinen","Miss. Laina""",female,26,0,0,STON/O2. 3101282,7.925,,S
3,4,1,1,"""Futrelle","Mrs. Jacques Heath (Lily May Peel)""",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"""Allen","Mr. William Henry""",male,35,0,0,373450,8.05,,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"""Montvila","Rev. Juozas""",male,27,0,0,211536,13,,S
887,888,1,1,"""Graham","Miss. Margaret Edith""",female,19,0,0,112053,30,B42,S
888,889,0,3,"""Johnston","Miss. Catherine Helen """"Carrie""""""",female,,1,2,W./C. 6607,23.45,,S
889,890,1,1,"""Behr","Mr. Karl Howell""",male,26,0,0,111369,30,C148,C


### Remoção de atributos sem significado na análise

In [9]:
# PassengerId é condizente com a própria numeração na tabela e único a cada um
dados = dados.drop(columns=['PassengerId'])

# Nome é um atributo sempre único e sem impacto na sobrevivência do passageiro
dados = dados.drop(columns=['Surname', 'Name'])

# Ticket é provavelmente irrelevante (supondo que não reflete fatores como localização no navio e outros)
dados = dados.drop(columns=['Ticket'])


# Apesar de estarem sendo desconsiderados, os atributos a seguir tiveram métodos de codificação incluídas no arquivo

# Cabine, apesar que pode ser relevante, tem uma taxa de 77% dos dados nulos nesse atributo, sendo difícil então o considerar
dados = dados.drop(columns=['Cabin'])

# O local de embarque não deve impactar a sobrevivência do passageiro, além de que uma maioria absoluta (72%) vieram do mesmo porto
dados = dados.drop(columns=['Embarked'])

### Codificação de atributos

##### Atributos binários

In [10]:
cols_label_encode = ['Sex']
dados[cols_label_encode] = dados[cols_label_encode].apply(LabelEncoder().fit_transform)

##### Codificação do atributo 'Embarked'

In [11]:
#for i in range(len(dados)):
#    if   dados['Embarked'][i] == "C":
#        dados.loc[i, 'Embarked'] = 0
#    elif dados['Embarked'][i] == "Q":
#        dados.loc[i, 'Embarked'] = 1
#    elif dados['Embarked'][i] == "S":
#        dados.loc[i, 'Embarked'] = 2

##### Codificação do atributo 'Cabin'

In [12]:
# Supostamente as cabines estão agrupadas relativo à sua proximidade, que seria refletido pela letra inicial, logo sendo possível considerá-las

#for i in range(len(dados)):
#    if  ( len(dados['Cabin'][i]) > 0 ):
#        cabine = dados['Cabin'][i][0]
#        #print(cabine, "    ", ord(cabine))
#        dados.loc[i, 'Cabin'] = ( ord(cabine) - 65 )

### Tratamento de dados vazios

In [13]:
dados

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22,1,0,7.25
1,1,1,0,38,1,0,71.2833
2,1,3,0,26,0,0,7.925
3,1,1,0,35,1,0,53.1
4,0,3,1,35,0,0,8.05
...,...,...,...,...,...,...,...
886,0,2,1,27,0,0,13
887,1,1,0,19,0,0,30
888,0,3,0,,1,2,23.45
889,1,1,1,26,0,0,30


In [14]:
# Obter moda e inserir no lugar
for column in dados.columns:
    grupo = dados[column]

    # Determinar moda
    moda = Counter(grupo).most_common(1)[0][0]
    if not moda:
        moda = Counter(grupo).most_common(2)[1][0]
    #print("Moda de ", column, "é ", moda)

    # Preencher dados vazios
    dados[column] = dados[column].fillna(moda)
    for linha in range(len(dados)):
        if ( type(dados[column][linha]) == str ):
            if not ( len(dados[column][linha]) > 0 ):
                #print ("Str: ", linha, dados[column][linha])
                dados.loc[linha, column] = ( moda )
                #print ("Str (new): ", linha, dados[column][linha])
        elif ( type(dados[column][linha]) == int ):
            if not ( dados[column][linha] >= 0 ):
                #print ("Int: ", linha, dados[column][linha])
                dados.loc[linha, column] = ( moda )
                #print ("Int (new): ", linha, dados[column][linha])

### Dados totalmente codificados

In [15]:
#for i in range(len(dados)):
#    for j in dados.columns:
#        print ( type(dados[j][i]) )
#        if ( type(dados[j][i]) == str ):
#            print ( dados[j][i] )
dados

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22,1,0,7.25
1,1,1,0,38,1,0,71.2833
2,1,3,0,26,0,0,7.925
3,1,1,0,35,1,0,53.1
4,0,3,1,35,0,0,8.05
...,...,...,...,...,...,...,...
886,0,2,1,27,0,0,13
887,1,1,0,19,0,0,30
888,0,3,0,24,1,2,23.45
889,1,1,1,26,0,0,30


### Separação de dados para treino e teste

In [16]:
# Separação de dados de atributo da conclusão desses
x_prev = dados.drop(columns=['Survived'])    # Atributos
y_classe = dados.iloc[:, [0]]                # Conclusão

# Separar dados em treino e teste
# (parte é usada para treinar a árvore, os demais servem para testes futuros do modelo)
# x representa os atributos
# y representa a conclusão
# treino são os dados de treino
# teste são os dados de teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x_prev, y_classe, test_size = 0.20, random_state = 42)

In [21]:
x_teste

,Pclass,Sex,Age,SibSp,Parch,Fare
709,3,1,24,1,1,15.2458
439,2,1,31,0,0,10.5
840,3,1,20,0,0,7.925
720,2,0,6,0,1,33
39,3,0,14,1,0,11.2417
...,...,...,...,...,...,...
433,3,1,17,0,0,7.125
773,3,1,24,0,0,7.225
25,3,0,38,1,5,31.3875
84,2,0,17,0,0,10.5


### Salvar dados formatados em novo arquivo

In [18]:
with open('Dados/Titanic.pkl', mode = 'wb') as f:
    pickle.dump([x_treino, x_teste, y_treino, y_teste], f)